In [1]:
import os
from timeit import default_timer as timer
from datetime import datetime

import numpy as np

#os.environ["PYSPARK_PYTHON"] = "/usr/local/anaconda3/bin/python"
#os.environ["PYSPARK_DRIVER_PYTHON"] = "/usr/local/anaconda3/bin/python"

#os.environ['HADOOP_LOG_DIR'] = '/Users/elissandro/Projetos/Especializacao/MDMR/logs'
#os.environ['HADOOP_CONF_DIR'] = '/Users/elissandro/Projetos/Especializacao/MDMR/conf/spark'
#os.environ['HADOOP_USER_NAME'] = 'elissandro'

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1600992860161_0003,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
# Ajustando locale da JVM para en-US, estava misturando os locale e ficando en-BR...
# Fonte: https://stackoverflow.com/questions/55246080/pyspark-stopwordsremover-parameter-locale-given-invalid-value
locale = spark.sparkContext._jvm.java.util.Locale
locale.setDefault(locale.forLanguageTag("en-US"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover

def load_file(filePath, type, delimiter = None):
    if type == 'text':
        df = spark.read.load(filePath, format='text')
    else:
        df = spark.read.csv(filePath, inferSchema=True, header=True, sep=delimiter)
    return df

def concat_all_reviews_v1(reviews):
    """
        Concatena todas as reviews (linhas no DF) em um RDD no formato (1, TEXTO).
        Retorna um RDD
    """
    rdd = reviews.rdd.map(lambda r: (1, r[0])).reduceByKey(lambda a, b: a + ' ' + b)
    return rdd

def concat_all_reviews_v2(reviews):
    """
        Concatena todas as reviews (linhas no DF) em um texto único.
        Retorna uma string
    """
    rdd = reviews.rdd.flatMap(lambda v: v[0].split()).reduce(lambda a, b: a + ' ' + b)
    return rdd

def tokenize_reviews(reviews):
    tokenization = Tokenizer(inputCol='review', outputCol='review_words')
    return tokenization.transform(reviews)

def remove_stop_words_reviews(reviews):
    stopword_removal = StopWordsRemover(inputCol='review_words',outputCol='review_words_refined')
    return stopword_removal.transform(reviews)

def counter_words_rdd_flatMap(df, index):
    """
        Filtra e conta palavras usando RDD flatMap
    """
    filtered = refined_reviews \
                .rdd \
                .flatMap(lambda v: [(w, 1) for w in v[index] if w != '' and len(w) > 3]) \
                .reduceByKey(lambda c1, c2: c1 + c2)
    return filtered

def counter_words_sql_expr(df, colunmName):
    """
        Filtra e conta palavras usando SparkSQL
    """
    from pyspark.sql.functions import expr
    
    filter_expr = f"filter({colunmName}, x -> x != '' and char_length(x) > 3)"
    filtered = df.withColumn(colunmName, expr(filter_expr)).rdd.flatMap(lambda v: v[0])

    return filtered.groupBy(colunmName).count()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
hdfs_folder = 'hdfs://192.168.1.37:9000/trabalho-final'

#local_path = '/Users/elissandro/Projetos/Especializacao/MDMR/logs/exec_logs'
#if not os.path.exists(local_path):
#    os.makedirs(local_folder)
    
#exec_logs_folder = f"file://{local_path}"

#from pathlib import Path
#Path("local_folder").mkdir(parents=True, exist_ok=True)

now = datetime.now()

files = ["hdfs://ip-172-31-81-222.ec2.internal:8020/user/mobile_test_2.tsv"]

lines = []
exec_info = []
execs_per_file = 10

for f in files:    
    #content = load_file(f"{hdfs_folder}/{f}", type='tsv', delimiter='\t')
    for r in np.arange(0, execs_per_file):
        print("Round " + str(r))
        
        print(f"Loading file: {f}")
        content = load_file(f, type='tsv', delimiter='\t')

        start = timer()
        print("Removing empty reviews...")
        content = content.select('review_body').filter(content.review_body != '')
        
        print("Concating content...")
        all_reviews_rdd = concat_all_reviews_v1(content)
        
        print("Creating DF to Mlib...")
        # Cria DataFrame para uso com a Mlib
        all_reviews_df = spark.createDataFrame(data=all_reviews_rdd, schema=['id', 'review'])
        
        print("Tokenization...")
        all_reviews_df_tokenized = tokenize_reviews(all_reviews_df)
        
        print("Removing StopWords...")
        refined_reviews = remove_stop_words_reviews(all_reviews_df_tokenized)
        
        print("Filter and counter Words...")
        filtered = counter_words_rdd_flatMap(refined_reviews, 3)
        
        end = timer()
        exec_time = end - start
        l = (f, str(r), "{:.4f}".format(exec_time))
        lines.append(l)
        print(f"Execution time:{exec_time} sec")
        print("---------------------------")
        
    print("Writing exec logs...")
    logs_execs = spark.createDataFrame(data=lines, schema=['file', 'round', 'time'])
    # logs_execs.write.csv(f"{exec_logs_folder}/exec_{f}_{now.strftime('%Y%m%d-%H%M%S')}.csv")
    
    log_file = f"s3://aws-emr-resources-188023916529-us-east-1/notebooks/e-8IEIV3G0OF4JFD9X5P7TBISY6/exec_{now.strftime('%Y%m%d-%H%M%S')}.csv"
    logs_execs.repartition(1).write.csv(path=log_file, mode="append", header="true")
    
    # logs_execs.coalesce(1).write.format("text").option("header", "false").mode("append").save(log_file)


filtered.take(10)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Loading file: hdfs://ip-172-31-81-222.ec2.internal:8020/user/mobile_test_2.tsv
Round 0
Removing empty reviews...
Concating content...
Creating DF to Mlib...
Tokenization...
Removing StopWords...
Filter and counter Words...
Execution time:3.9530566000012186 sec
---------------------------
Loading file: hdfs://ip-172-31-81-222.ec2.internal:8020/user/mobile_test_2.tsv
Round 1
Removing empty reviews...
Concating content...
Creating DF to Mlib...
Tokenization...
Removing StopWords...
Filter and counter Words...
Execution time:0.5652271480012132 sec
---------------------------
Loading file: hdfs://ip-172-31-81-222.ec2.internal:8020/user/mobile_test_2.tsv
Round 2
Removing empty reviews...
Concating content...
Creating DF to Mlib...
Tokenization...
Removing StopWords...
Filter and counter Words...
Execution time:0.5553559740001219 sec
---------------------------
Loading file: hdfs://ip-172-31-81-222.ec2.internal:8020/user/mobile_test_2.tsv
Round 3
Removing empty reviews...
Concating content...

In [18]:
spark.stop()

VBox()

An error was encountered:
Invalid status code '404' from https://172.31.81.222:18888/sessions/0 with error payload: {"msg":"Session '0' not found."}


In [2]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
content = load_file(f, type='tsv', delimiter='\t')
content.printSchema()
        

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- <!DOCTYPE html>: string (nullable = true)